In [37]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
from pymongo import MongoClient
import nltk

In [33]:
url_base = 'https://www.conjugacao.com.br/verbos-populares'
urls_paginas = [url_base] + [f'{url_base}/{i}/' for i in range(2, 51)]

In [35]:
def get_verbos_pagina(url_pagina):
    print(f'Buscando verbos da pagina {url_pagina}')
    # Espera 5 segundos antes de buscar a proxima pagina
    time.sleep(5)
    page = requests.get(url_pagina)
    if page.status_code == 200:
        soap = BeautifulSoup(page.text, 'html.parser')
        div_content = soap.find('div', {'id': 'content'})
        lis = div_content.findChildren('li')
        verbos_pagina = [li.getText() for li in lis]
        return verbos_pagina
    else:
        return []

In [36]:
verbos_por_paginas = [get_verbos_pagina(url) for url in urls_paginas]

Buscando verbos da pagina https://www.conjugacao.com.br/verbos-populares
Buscando verbos da pagina https://www.conjugacao.com.br/verbos-populares/2/
Buscando verbos da pagina https://www.conjugacao.com.br/verbos-populares/3/
Buscando verbos da pagina https://www.conjugacao.com.br/verbos-populares/4/
Buscando verbos da pagina https://www.conjugacao.com.br/verbos-populares/5/
Buscando verbos da pagina https://www.conjugacao.com.br/verbos-populares/6/
Buscando verbos da pagina https://www.conjugacao.com.br/verbos-populares/7/
Buscando verbos da pagina https://www.conjugacao.com.br/verbos-populares/8/
Buscando verbos da pagina https://www.conjugacao.com.br/verbos-populares/9/
Buscando verbos da pagina https://www.conjugacao.com.br/verbos-populares/10/
Buscando verbos da pagina https://www.conjugacao.com.br/verbos-populares/11/
Buscando verbos da pagina https://www.conjugacao.com.br/verbos-populares/12/
Buscando verbos da pagina https://www.conjugacao.com.br/verbos-populares/13/
Buscando ve

In [44]:
todos_verbos = [verbo for verbos_pagina in verbos_por_paginas for verbo in verbos_pagina]

In [45]:
len(todos_verbos)

5000

In [41]:
def get_collection_verbos():
    client = MongoClient()
    tcc_pucmg_db = client['tcc-pucmg']
    return tcc_pucmg_db.verbos

In [42]:
stemmer = nltk.stem.RSLPStemmer()
verbos = get_collection_verbos()

In [43]:
def criar_documento(verbo):
    return {
        'verbo': verbo,
        'verbo_stemmed': stemmer.stem(verbo)
    }

In [46]:
documentos_verbos = [criar_documento(v) for v in todos_verbos]

In [47]:
verbos.insert_many(documentos_verbos)